In [ ]:
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00


In [ ]:
# === Patched: Load NusaX-MT (ind→mad) from GitHub CSVs instead of HF scripted dataset ===
from datasets import load_dataset, DatasetDict, Dataset

URLS = {
    "train": "https://raw.githubusercontent.com/IndoNLP/nusax/main/datasets/mt/train.csv",
    "validation": "https://raw.githubusercontent.com/IndoNLP/nusax/main/datasets/mt/valid.csv",
    "test": "https://raw.githubusercontent.com/IndoNLP/nusax/main/datasets/mt/test.csv",
}
SRC_COL, TGT_COL = "indonesian", "madurese"   # Indonesia -> Madurese

raw = load_dataset("csv", data_files=URLS)

def to_pair(ex):
    return {"text_1": ex[SRC_COL], "text_2": ex[TGT_COL]}

def dedup_ds(ds, cols=("text_1","text_2")):
    # convert -> pandas -> drop_duplicates -> back to Dataset
    df = ds.to_pandas()
    df = df.dropna(subset=list(cols))
    df = df[df[cols[0]].astype(str).str.strip() != ""]
    df = df[df[cols[1]].astype(str).str.strip() != ""]
    df = df.drop_duplicates(subset=list(cols))
    return Dataset.from_pandas(df, preserve_index=False)

mdr = DatasetDict()
for sp in raw.keys():
    cur = raw[sp].map(
        to_pair,
        remove_columns=[c for c in raw[sp].column_names if c not in (SRC_COL, TGT_COL)],
    )
    # bersih + dedup via pandas
    cur = dedup_ds(cur, ("text_1","text_2"))
    mdr[sp] = cur

print(mdr)
print(mdr["train"][0])

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['indonesian', 'madurese', 'text_1', 'text_2'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['indonesian', 'madurese', 'text_1', 'text_2'],
        num_rows: 100
    })
    test: Dataset({
        features: ['indonesian', 'madurese', 'text_1', 'text_2'],
        num_rows: 400
    })
})
{'indonesian': 'Nikmati cicilan 0% hingga 12 bulan untuk pemesanan tiket pesawat air asia dengan kartu kredit bni!', 'madurese': 'Nikmati cecelan 0% sampek 12 bulen ghebey pamessenan karcis kapal air asia bik kartu kredit bni!', 'text_1': 'Nikmati cicilan 0% hingga 12 bulan untuk pemesanan tiket pesawat air asia dengan kartu kredit bni!', 'text_2': 'Nikmati cecelan 0% sampek 12 bulen ghebey pamessenan karcis kapal air asia bik kartu kredit bni!'}


In [ ]:
train = mdr["train"]
val = mdr["validation"]
test = mdr["test"]

In [ ]:
# ===================== 1) Setup =====================
from transformers import T5Tokenizer, MT5ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate, numpy as np
from datasets import Dataset  # kalau perlu, tergantung cara load dataset

CHECKPOINT = "indonlp/cendol-mt5-small-inst"   # model instruksi mT5 (base version)
PREFIX = "translate Madurese to Indonesian: "  # <-- ARAH MADURA -> INDONESIA (konsisten!)

tokenizer = T5Tokenizer.from_pretrained(CHECKPOINT)
model = MT5ForConditionalGeneration.from_pretrained(CHECKPOINT)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Penting untuk model T5/MT5: pastikan token2 ini terset
model.config.decoder_start_token_id = tokenizer.pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

In [ ]:
# ===================== 2) Preprocess =====================
def preprocess_function(examples):
    # text_1 = sumber (Indonesia), text_2 = target (Madura) - Need to swap for Madura -> Indonesia
    # Corrected: text_1 = Madura, text_2 = Indonesia
    inputs = [PREFIX + x for x in examples["text_2"]] # Use text_2 (Madurese) as input
    targets = examples["text_1"] # Use text_1 (Indonesian) as target

    # Tokenisasi dengan target
    model_inputs = tokenizer(
        inputs, max_length=64, truncation=True  # Reduced max_length
    )
    labels = tokenizer(
        targets, max_length=64, truncation=True  # Reduced max_length
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# ===================== 3) Metrik =====================
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds  = [p.strip() for p in preds]
    labels = [[l.strip()] for l in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    # filter out invalid token IDs before decoding
    preds = np.where((preds >= 0) & (preds < tokenizer.vocab_size), preds, tokenizer.pad_token_id)
    # decode
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # ganti -100 jadi pad_id sebelum decode labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # BLEU
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    out = {"bleu": result["score"]}

    # panjang prediksi (dlm token, non-pad)
    pred_lens = [np.count_nonzero(p != tokenizer.pad_token_id) for p in preds]
    out["gen_len"] = float(np.mean(pred_lens)) if len(pred_lens) else 0.0
    return out

In [ ]:
# ===================== 4) Training =====================
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

training_args = Seq2SeqTrainingArguments(
    output_dir="cendol-mt5-mad2ind",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=4,  # Further reduced batch size
    per_device_eval_batch_size=4,   # Further reduced batch size
    gradient_accumulation_steps=8, # Increase gradient accumulation
    weight_decay=0.01,
    num_train_epochs=25,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True,
    generation_max_length=64,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
    gradient_checkpointing=True, # Enable gradient checkpointing
)

# Tokenize the datasets
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_val = val.map(preprocess_function, batched=True)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

#8e273a83f932e54c56230b4e303a2f335701ad7b

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/tmp/ipython-input-4149100831.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: najladhia259 (najladhia259-airlangga) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.727295,31.116514,33.930000
2,No log,1.684941,33.198720,34.040000
3,No log,1.669327,34.386899,34.390000
4,1.794900,1.643194,35.221094,34.110000
5,1.794900,1.648405,36.515771,34.330000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
# ===================== 5) Inference helper =====================
import torch
@torch.no_grad()
def translate_batch(texts, max_new_tokens=64, num_beams=4):
    # texts = list of raw Madurese sentences
    prompts = [PREFIX + t for t in texts]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model.generate(
        **{k: v.to(model.device) for k, v in inputs.items()},
        max_new_tokens=max_new_tokens,
        num_beams=num_beams,
        early_stopping=True
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
# ===================== 6) Sanity-check di data test =====================
import random

def translate_and_compare_samples(raw_test, num_samples=5):
    """
    raw_test: dict/Dataset dengan field 'text_1' (Madura), 'text_2' (Indonesia)
    """
    idxs = random.sample(range(len(raw_test['text_1'])), min(num_samples, len(raw_test['text_1'])))
    for i in idxs:
        src = raw_test['text_1'][i]   # Madura
        ref = raw_test['text_2'][i]   # Indonesia (ground truth)
        hyp = translate_batch([src])[0]
        print(f"[{i}] SRC(Madura) : {src}")
        print(f"    HYP(Indo)    : {hyp}")
        print(f"    REF(Indo)    : {ref}")
        print("-"*60)


In [ ]:
# Contoh pakai:
translate_and_compare_samples(test, num_samples=5)

# ===================== 7) Single example =====================
# contoh teks Madura -> Indonesia
teks_madura = "kaulâh ngakan 7 es krim" # Corrected example text to be in Madurese
print(translate_batch([teks_madura])[0])

In [ ]:
# ===================== 8) Save model and tokenizer =====================
output_dir = "mad2ind_translation_model"

# Save the fine-tuned model
trainer.model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")